<a href="https://colab.research.google.com/github/riphunter7001x/Smart-Compliance-Assistant-using-Self-RAG/blob/main/graph_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --no-cache-dir -q langchain langchain-experimental langchain-community langchain-groq langchain-neo4j neo4j


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 325.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 141.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 346.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 343.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 321.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 289.6 MB/s eta 0:00:00


In [38]:
# graph DB creds
from google.colab import userdata
NEO4J_URI=userdata.get('NEO4J_URI')
NEO4J_USERNAME=userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD=userdata.get('NEO4J_PASSWORD')


In [39]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


## setting up Neo4j instance

In [13]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(
    url=os.environ.get("NEO4J_URI"),
    username=os.environ.get("NEO4J_USERNAME"),
    password=os.environ.get("NEO4J_PASSWORD")
)

<ipython-input-13-bb0351c61dcc>:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [5]:
# graph = Neo4jGraph(url="your_neo4j_url", username="your_username", password="your_password")

In [40]:
graph

## loading LLM model

In [7]:
from google.colab import userdata
GROQ_API_KEY=userdata.get('GROQ_API_KEY')

os.environ["GROQ_API_KEY"] = GROQ_API_KEY
from langchain_groq import ChatGroq

In [8]:
llm = ChatGroq(model="llama3-70b-8192")

In [9]:
llm.invoke("tell me joke ")

AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you laugh!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 14, 'total_tokens': 47, 'completion_time': 0.100800885, 'prompt_time': 0.003429756, 'queue_time': 0.020525482, 'total_time': 0.104230641}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_7ab5f7e105', 'finish_reason': 'stop', 'logprobs': None}, id='run-f92911d4-42ac-433c-a369-2fb14d0486a9-0', usage_metadata={'input_tokens': 14, 'output_tokens': 33, 'total_tokens': 47})

# loading the data

In [ ]:
from langchain_core.documents import Document

In [ ]:
text = """
Elon Reeve Musk (born June 28, 1971) is a businessman and political figure known for his key roles in the automotive company Tesla,
Inc. and the space company SpaceX. He is also known for his ownership of the technology company X Corp. and his role in the founding of the Boring
Company, xAI, Neuralink, and OpenAI. Musk is the wealthiest individual in the world; as of January 2025, Forbes estimates his net worth to be US$426 billion.

A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria. At the age of 18 he
immigrated to Canada, acquiring its citizenship through his Canadian-born mother, Maye. Two years later, he matriculated at Queen's University in
Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics. He moved to California
in 1995 to attend Stanford University but never enrolled in classes, and with his brother Kimbal co-founded the online city guide software company
Zip2. The startup was acquired by Compaq for $307 million in 1999. That same year, Musk co-founded X.com, a direct bank. X.com merged with
Confinity in 2000 to form PayPal. In 2002, Musk acquired United States citizenship, and that October eBay acquired PayPal for $1.5 billion.
Using $100 million of the money he made from the sale of PayPal, Musk founded SpaceX, a spaceflight services company, in 2002.

In 2004, Musk was an early investor in electric vehicle manufacturer Tesla Motors, Inc. (later Tesla, Inc.), providing most of the initial
financing and assuming the position of the company's chairman. He later became the product architect and, in 2008, the CEO. In 2006, Musk
helped create SolarCity, a solar energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop
high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following
year Musk co-founded Neuralink, a neurotechnology company developing brain–computer interfaces, and the Boring Company, a tunnel construction
company. In 2018, the U.S. Securities and Exchange Commission (SEC) sued Musk, alleging he falsely announced that he had secured funding for a
private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine. In 2022, he acquired
Twitter for $44 billion, merged the company into his newly-created X Corp., and rebranded the service as X the following year. In 2023, Musk
founded xAI, an artificial intelligence company.

"""

In [ ]:
docs = [Document(page_content=text)]

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [ ]:
llm_transformer = LLMGraphTransformer(llm=llm)

# converted text to nodes

In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(docs)

In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Tesla, Inc.', type='Company', properties={}), Node(id='Spacex', type='Company', properties={}), Node(id='X Corp.', type='Company', properties={}), Node(id='The Boring Company', type='Company', properties={}), Node(id='Xai', type='Company', properties={}), Node(id='Neuralink', type='Company', properties={}), Node(id='Openai', type='Company', properties={}), Node(id='University Of Pretoria', type='University', properties={}), Node(id="Queen'S University", type='University', properties={}), Node(id='University Of Pennsylvania', type='University', properties={}), Node(id='Stanford University', type='University', properties={}), Node(id='Zip2', type='Company', properties={}), Node(id='Compaq', type='Company', properties={}), Node(id='X.Com', type='Company', properties={}), Node(id='Confinity', type='Company', properties={}), Node(id='Paypal', type='Company', properties={}), Node(id='Ebay', type='Company

In [ ]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Tesla, Inc.', type='Company', properties={}),
 Node(id='Spacex', type='Company', properties={}),
 Node(id='X Corp.', type='Company', properties={}),
 Node(id='The Boring Company', type='Company', properties={}),
 Node(id='Xai', type='Company', properties={}),
 Node(id='Neuralink', type='Company', properties={}),
 Node(id='Openai', type='Company', properties={}),
 Node(id='University Of Pretoria', type='University', properties={}),
 Node(id="Queen'S University", type='University', properties={}),
 Node(id='University Of Pennsylvania', type='University', properties={}),
 Node(id='Stanford University', type='University', properties={}),
 Node(id='Zip2', type='Company', properties={}),
 Node(id='Compaq', type='Company', properties={}),
 Node(id='X.Com', type='Company', properties={}),
 Node(id='Confinity', type='Company', properties={}),
 Node(id='Paypal', type='Company', properties={}),
 Node(id='Ebay', type='Company', p

In [ ]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla, Inc.', type='Company', properties={}), type='OWNS', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Company', properties={}), type='OWNS', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X Corp.', type='Company', properties={}), type='OWNS', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='The Boring Company', type='Company', properties={}), type='OWNS', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Xai', type='Company', properties={}), type='OWNS', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Neuralink', type='Company', properties={}), type='OWNS', properti

In [ ]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Tesla, Inc.', type='Company', properties={}), Node(id='Spacex', type='Company', properties={}), Node(id='X Corp.', type='Company', properties={}), Node(id='The Boring Company', type='Company', properties={}), Node(id='Xai', type='Company', properties={}), Node(id='Neuralink', type='Company', properties={}), Node(id='Openai', type='Company', properties={}), Node(id='University Of Pretoria', type='University', properties={}), Node(id="Queen'S University", type='University', properties={}), Node(id='University Of Pennsylvania', type='University', properties={}), Node(id='Stanford University', type='University', properties={}), Node(id='Zip2', type='Company', properties={}), Node(id='Compaq', type='Company', properties={}), Node(id='X.Com', type='Company', properties={}), Node(id='Confinity', type='Company', properties={}), Node(id='Paypal', type='Company', properties={}), Node(id='Ebay', type='Company'

## bulk data insertion

In [ ]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [ ]:
graph.query(movie_query)

[]

## get DB schema

In [14]:
graph.refresh_schema()
print(graph.schema)

Node properties:
person {born: INTEGER, name: STRING}
movie {title: STRING, released: INTEGER}
Movie {title: STRING, released: DATE, id: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:person)-[:ACTED_IN]->(:movie)
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


# setting up QA chain

In [15]:
from langchain.chains import GraphCypherQAChain


In [17]:
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x786926e5f790>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7869269903d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x786958e306d0>, model_name='llama3-70b-

In [18]:
response=chain.invoke({"query":"Who was the director of the moview GoldenEye"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:DIRECTED]-(p:Person) RETURN p.name
Full Context:
[{'p.name': 'Martin Campbell'}]

> Finished chain.


{'query': 'Who was the director of the moview GoldenEye',
 'result': 'Martin Campbell was the director of the movie GoldenEye.'}

In [19]:
response=chain.invoke({"query":"tell me the genre of th movie GoldenEye"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})-[:IN_GENRE]->(g:Genre) RETURN g.name
Full Context:
[{'g.name': 'Adventure'}, {'g.name': 'Action'}, {'g.name': 'Thriller'}]

> Finished chain.


{'query': 'tell me the genre of th movie GoldenEye',
 'result': 'The genre of the movie GoldenEye is Adventure, Action, Thriller.'}

In [20]:
response=chain.invoke({"query":"Who was the director in movie Casino"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name
Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director in movie Casino',
 'result': 'Martin Scorsese was the director in the movie Casino.'}

In [21]:
response=chain.invoke({"query":"Which movie were released in 2008"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.released = 2008 RETURN m.title;
Full Context:
[]

> Finished chain.


{'query': 'Which movie were released in 2008',
 'result': "I don't know the answer."}

In [25]:
response=chain.invoke({"query":"Which movie were released in 2015"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.released = 2015 RETURN m.title;
Full Context:
[]

> Finished chain.


{'query': 'Which movie were released in 2015',
 'result': "I don't know the answer."}

In [26]:
response=chain.invoke({"query":"Give me the list of movie having imdb rating more than 8"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.imdbRating > 8 RETURN m.title, m.imdbRating ORDER BY m.imdbRating DESC;
Full Context:
[{'m.title': 'Shawshank Redemption, The', 'm.imdbRating': 9.3}, {'m.title': 'Pulp Fiction', 'm.imdbRating': 8.9}, {'m.title': 'Star Wars: Episode IV - A New Hope', 'm.imdbRating': 8.7}, {'m.title': 'Seven (a.k.a. Se7en)', 'm.imdbRating': 8.6}, {'m.title': 'Usual Suspects, The', 'm.imdbRating': 8.6}, {'m.title': 'Léon: The Professional (a.k.a. The Professional) (Léon)', 'm.imdbRating': 8.6}, {'m.title': 'Braveheart', 'm.imdbRating': 8.4}, {'m.title': 'Toy Story', 'm.imdbRating': 8.3}, {'m.title': 'Taxi Driver', 'm.imdbRating': 8.3}, {'m.title': 'Hoop Dreams', 'm.imdbRating': 8.3}]

> Finished chain.


{'query': 'Give me the list of movie having imdb rating more than 8',
 'result': 'Here are the movies with an IMDb rating more than 8: Shawshank Redemption, The, Pulp Fiction, Star Wars: Episode IV - A New Hope, Seven (a.k.a. Se7en), Usual Suspects, The, Léon: The Professional (a.k.a. The Professional) (Léon), Braveheart, Toy Story, Taxi Driver, Hoop Dreams.'}

In [35]:
response=chain.invoke({"query": "How many unique artists are there ?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN COUNT(DISTINCT p) AS unique_artists
Full Context:
[{'unique_artists': 1239}]

> Finished chain.


{'query': 'How many unique artists are there ?',
 'result': 'There are 1239 unique artists.'}

SyntaxError: illegal target for annotation (<ipython-input-36-27dd68d80ee7>, line 1)

In [37]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:person {name: "Tom Hanks"})-[:ACTED_IN]->(m:movie) RETURN count(m)
Full Context:
[{'count(m)': 0}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in?',
 'result': "I don't know the answer."}